# Functions

1. Function returning its Value V, given v and the strike K of a European put option on the asset S, expiring after N periods.

In [76]:
def put_option(S0, v, K, N, p = 0.5):
    """
    Valuation of a European put option on asset S
    Parameters:
    - S: Initial price of asset 
    - v: risk free rate
    - K: Strike price
    - N: Periods
        
    Returns: Value of option

    """
    ## To calculate todays prices we use risk neutral probabilities under risk neutral measure
    u = (1+v) # up state
    d = (1-v) # down state
        
    # Calculate the asset prices at each period
    S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]

    # Calculate the option value
    V = [max(K - S[j], 0) for j in range(N + 1)] # value of put is strike - stock price

   # Calculate option values at earlier periods in the tree working backwards
    for n in range (N-1, -1, -1):
        V = [p * (max(K-S[k],0) + (1-p) * (max(K-S[k+1],0))) for k in range(n+1)]
     
    # return value at time 0
    return V[0]

1. A function which, given the strike K and value V of a European put option on the asset S, expiring after N periods, calibrates v to match this price.

In [81]:
def calibrate_v(K, V, N, S0):
    """
    Funtion that calibrated v to match price V

    Parameters:
    - S0: Initial price of asset 
    - V: value of option
    - K: Strike price
    - N: Periods
        
    Returns: Calibrated value of v

    """
    # Define the parameters for the multiperiod binomial asset model
    a = 0.0001  # lower bound of v
    b = 0.9999  # upper bound of v
    tol = 0.0001  # tolerance for convergence

    # Define the function V to be solved
    def f(v):
        u = 1 + v
        d = 1 - v
        S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]
        V_cal = [max(K - S[j], 0) for j in range(N + 1)]
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                V_cal[j] = (p * V_cal[j] + (1 - p) * V_cal[j + 1]) / (1 + r)
                S[j] = (1 + v) * S[j]
            del V_cal[-1]
        return V_cal[0] - V

    # Use binary search method to find the value of v that matches the option value
    while b - a > tol:
        c = (a + b) / 2
        if f(c) == 0:
            return c
        elif f(a) * f(c) < 0:
            b = c
        else:
            a = c

    return (a + b) / 2

3. A function which, given v, returns the expectation of max S_j

In [88]:
def maxS(v, N):
    """
    Calculate the expectation of the maximum value among a series of prices sj given parameter v.

    Parameters:
    - v: value of option
    - N: Periods
        
    Returns: Maximum S

    """
    # Define the parameters for the multiperiod binomial asset model
    S0 = 1.28065  # initial price of the asset
    u = 1 + v  # up factor
    d = 1 - v  # down factor
    p = 0.5  # risk-neutral probability

    # Calculate the asset prices at each period
    S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]

    # Calculate the maximum asset value over all periods
    max_S = max(S)

    # Calculate the expectation of the maximum asset value
    exp_max_S = max_S
    
    for i in range(N):
        exp_max_S = (p * max(u ** (N - i - 1) * S[i], d ** (N - i - 1) * S[i + 1]) +
                     (1 - p) * exp_max_S) / (1 + r)

    return exp_max_S



In [ ]:
def calculate_put_option_payoff(S, K):
    """
    Calculates the payoff of a European put option.

    Args:
    S (list): List of prices of the underlying asset at each period.
    K (float): Strike price of the put option.

    Returns:
    list: Payoff of the put option at each period.
    """
    option_payoff = [max(K - s, 0) for s in S]
    return option_payoff